In [1]:
# Parsing SR

In [2]:
!pip install pypdf2 docling -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/54

In [3]:
import pandas as pd
import PyPDF2
import glob
from docling.document_converter import DocumentConverter
from tqdm import tqdm
import json
import os
import re
os.environ["TORCH_CUDA_ARCH_LIST"] = "7.5"

In [4]:
from google.colab import files

uploaded = files.upload()
for file_name in uploaded.keys():
    print(f"File diunggah: {file_name}")

Saving asii_2023_sustainability_report.pdf to asii_2023_sustainability_report.pdf
File diunggah: asii_2023_sustainability_report.pdf


In [5]:
def get_pdf_page_count(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            return len(reader.pages)
    except Exception as e:
        print(f"An error occurred while counting pages: {e}")
        return -1

def extract_page(input_pdf_path, output_pdf_path, page_number):
    try:
        with open(input_pdf_path, 'rb') as input_pdf:
            reader = PyPDF2.PdfReader(input_pdf)
            writer = PyPDF2.PdfWriter()

            # Check if the requested page number is valid
            if page_number < 0 or page_number >= len(reader.pages):
                raise ValueError("Invalid page number.")

            # Extract the specific page
            writer.add_page(reader.pages[page_number])

            # Write to a new PDF file
            with open(output_pdf_path, 'wb') as output_pdf:
                writer.write(output_pdf)
                # print(f"Page {page_number + 1} extracted and saved to {output_pdf_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def save_to_json(data_list, json_file_path):
    try:
        with open(json_file_path, 'w') as json_file:
            json.dump(data_list, json_file, indent=4)
            # print(f"List saved to {json_file_path}")
    except Exception as e:
        print(f"An error occurred while saving the list to JSON: {e}")

def get_indicator(content):
    # Daftar indikator GRI yang valid
    valid_indicators = [
        '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10',
        '2-11', '2-12', '2-13', '2-14', '2-15', '2-16', '2-17', '2-18', '2-19',
        '2-20', '2-21', '2-22', '2-23', '2-24', '2-25', '2-26', '2-27', '2-28',
        '2-29', '2-30', '3-1', '3-2', '3-3', '201-1', '201-2', '201-3', '201-4',
        '202-1', '202-2', '203-1', '203-2', '204-1', '205-1', '205-2', '205-3',
        '206-1', '207-1', '207-2', '207-3', '207-4', '301-1', '301-2', '301-3',
        '302-1', '302-2', '302-3', '302-4', '302-5', '303-1', '303-2', '303-3',
        '303-5', '304-1', '304-2', '304-3', '304-4', '305-1', '305-2', '305-3',
        '305-4', '305-5', '305-6', '305-7', '306-1', '306-2', '306-3', '306-4',
        '306-5', '308-1', '308-2', '401-1', '401-2', '401-3', '402-1', '403-1',
        '403-2', '403-3', '403-4', '403-5', '403-6', '403-7', '403-8', '403-9',
        '403-10', '404-1', '404-2', '404-3', '405-1', '405-2', '406-1', '407-1',
        '408-1', '409-1', '410-1', '411-1', '413-1', '413-2', '414-1', '414-2',
        '415-1', '416-1', '416-2', '417-1', '417-2', '417-3', '418-1'
    ]

    # Pola regex untuk mencocokkan kode indikator GRI
    pattern = r'\b\d{1,3}-\d{1,2}(?=\D|$)'

    # Mencari semua kode yang cocok dengan pola
    found_codes = re.findall(pattern, content)

    # Memfilter kode yang valid
    disclosed_indicators = [code for code in found_codes if code in valid_indicators]

    return disclosed_indicators

In [6]:
file = "asii_2023_sustainability_report.pdf"

count = get_pdf_page_count(file)

results = []
for i in tqdm(range(count)):
    source = "temp.pdf"
    extract_page(file, source, i)
    converter = DocumentConverter()
    result = converter.convert(source)
    result_md = result.document.export_to_markdown()
    disclosed_gri = get_indicator(result_md+"405-1")
    results.append({
        "page_content": result_md,
        "metadata": {
            "page": i+1,
            "company": "PT Astra International Tbk",
            "ticker": "ASII",
            "disclosed_gri": disclosed_gri
        }
    })
    save_to_json(results, os.path.basename(file)[:-4]+'.json')

100%|██████████| 210/210 [19:23<00:00,  5.54s/it]
